[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elsamuel27/snapthon/blob/main/Snapthon.ipynb)

In [ ]:
#@title # Paso 1: Rellena el formulario a continuación y ejecuta todas las celdas

mult = "Video" #@param ["Video", "Playlist"]
link = "" #@param {type: "string"}
ft = "Audio" #@param ["Video", "Audio"]
format = "mp3" #@param ["mp3", "mp4"] {allow-input: true}
route = "/content/Descargados"
purification = False #@param {type: "boolean"}
#@markdown La purificación es un proceso que hace que la música sea más propensa a ser compatible con coches, casets, etc, pero aumenta mucho el tiempo de descarga. Actívala marcando la casilla de arriba. No tiene ningún efecto al momento de descargar vídeo, por lo que se desactivará automáticamente
zip = False #@param {type: "boolean"}
#@markdown Si vas a descargar una playlist con bastantes vídeos, es recomendable que marques la casilla de arriba para recibir tu contenido en un archivo ZIP
if ft != "Audio" and purification:
    purification = False

Para ejecutar todas las celdas haz click en `Entorno de ejecución > Ejecutar todas`, o pulsa `Ctrl + F9`

# Paso 2: Espera. En breve tendrás tus descargas listas

In [ ]:
#@title Actualizaciones

!pip install -U pip
!sudo apt update -y

In [ ]:
#@title Instalaciones
with open("requirements.txt", "a") as r:
  r.write("git+https://github.com/pytube/pytube.git\n")
  if purification:
    r.write("pydub\n")

!pip install -r requirements.txt
!sudo apt install zip -y

In [ ]:
#@title Importaciones

import os
from pytube import YouTube, Playlist
import threading
try:
    from google.colab import files
except:
    print("Uso fuera de Google Colab activado")
if purification:
    from pydub import AudioSegment

Una desventaja de que los nombres de los archivos descargados sean los títulos de los vídeos, es que pueden contener caracteres incompatibles con el sistema de archivos. Una solución sencilla que se me ha ocurrido es la siguiente: 

- Creamos una tupla con todos los caracteres no permitidos por Windows (Pues Linux admite unos cuántos más y para curar en salud mejor tener en cuenta más caracteres)
- En vez de usar el título directamente, lo almacenamos en otra variable, y acto seguido usamos el método `.replace()` de las cadenas de caracteres para reemplazar los caracteres no permitidos por nada, ergo, borrarlos

In [ ]:
char = ("\\", "/", ":", "*", "?", "\"", "<", ">", "|")

def descargar(video, ft):
  title = video.title
  for c in char:
    title = title.replace(c, "")
  file = f"{title}.{format}"

  if ft == "Video":
    yd = video.streams.get_highest_resolution()
    yd.download(route, file)
    print("¡Vídeo descargado! Hasta luego :D")
  elif ft == "Audio":
    yd = video.streams.get_audio_only()
    yd.download(route, file)
    print("¡Vídeo descargado! Hasta luego :D")

def purificar(entrada, salida):
  audio = AudioSegment.from_file(entrada)

  audio.export(salida, format='mp3')

In [ ]:
#@title Descargas en el entorno

if mult == "Video":
    yt = YouTube(link)
    descargar(yt, ft)
elif mult == "Playlist":
    hilos = []
    pl = Playlist(link)
    for video in pl.videos:
        hilo = threading.Thread(target=descargar, args=(video,ft))
        hilos.append(hilo)
        hilo.start()

    for hilo in hilos:
        hilo.join()

In [ ]:
#@title Purificación (Si se ha activado)
if purification:
    try:
        hilos.clear()
    except:
        hilos = []
    input_folder = "/content/Descargados"
    output_folder = "/content/bySnapthon"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name.rsplit('.', 1)[0] + '.mp3')
        
        hilo = threading.Thread(target=purificar, args=(input_file_path,output_file_path))
        hilos.append(hilo)
        hilo.start()

    for hilo in hilos:
        hilo.join()

    os.system("rm -rf ./Descargados")
else:
  os.system("mv ./Descargados ./bySnapthon")

In [ ]:
#@title Descarga

if zip:
  os.system("zip -r /content/bySnapthon.zip ./bySnapthon")
  try:
    files.download("/content/bySnapthon.zip")
  except:
    print("Uso fuera de Google Colab detectado.")
else:
  try:
    for video in os.listdir("/content/bySnapthon"):
      files.download(f"/content/bySnapthon/{video}")
  except:
    print("Uso fuera de Google Colab detectado")

# Paso 3: Disfruta el contenido descargado